<a href="https://colab.research.google.com/github/dht7166/NLP_tweet/blob/master/kaggle_tweet_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle: Tweet Real or Not?

In [0]:
from google.colab import drive
drive.mount('/content/drive')
mydir = 'drive/My Drive/kaggle_nlp_tweet/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import keras
from keras.layers import Embedding,LSTM, Dense, Dropout
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import  Constant

## Read and Prepare the database

In [0]:
import pandas as pd
import os
db = pd.read_csv(os.path.join(mydir,'train.csv'))
db

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [0]:
# We can combine keyword, location into the actual text and ignore punctuation

tweet = []
target = []
max_length = 0
for _, row in db.iterrows():
  target.append(row['target'])
  text = row['text']
  text = text + ' ' + str(row['keyword'])
  text = text + ' ' + str(row['location'])
  # We would also remove any links or tags
  list_word = text.split()
  new_list = [word for word in list_word if ('http' not in word) and ('@' not in word) ]
  max_length = max([max_length,len(new_list)-1])
  tweet.append(' '.join(new_list))

token = Tokenizer()
token.fit_on_texts(tweet)

X = token.texts_to_sequences(tweet)
X = pad_sequences(X,maxlen=max_length)
X = np.asarray(X)
Y = np.asarray(target)





print("MAX STRING LENGTH ", max_length)
print("VOCAB SIZE",len(token.word_index)+1)


MAX STRING LENGTH  34
VOCAB SIZE 17711


## Create da model




In [0]:
# Build the model
model = keras.models.Sequential()
# We have word 2 vec pretrained so no need to retrain embedding layer
model.add(Embedding(len(token.word_index)+1,300,input_length = max_length,name = "Embedding"))
model.add(LSTM(300,name="LSTM"))
model.add(Dense(50,activation='relu',name="Dense"))
model.add(Dense(1,activation='sigmoid',name = "Out"))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.summary()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding (Embedding)        (None, 34, 300)           5313300   
_________________________________________________________________
LSTM (LSTM)                  (None, 300)               721200    
_________________________________________________________________
Dense (Dense)                (None, 50)                15050     
_________________________________________________________________
Out (Dense)                  (None, 1)                 51        
Total params: 6,049,601
Trainable params: 6,049,601
Non-trainable params: 0
_________________________________________________________________


In [0]:
ckpt_val = keras.callbacks.ModelCheckpoint(os.path.join(mydir,'best_val_acc.h5'),
                                                 monitor='val_acc', verbose=1, save_best_only=True)
ckpt_acc = keras.callbacks.ModelCheckpoint(os.path.join(mydir,'best_acc.h5'),
                                                 monitor='acc', verbose=1, save_best_only=True)

model.fit(X,Y,epochs=20,validation_split=0.2, callbacks=[ckpt_val,ckpt_acc])




Train on 6090 samples, validate on 1523 samples
Epoch 1/20





6090/6090 [==============================] - 69s 11ms/step - loss: 0.5260 - acc: 0.7366 - val_loss: 0.5132 - val_acc: 0.7636

Epoch 00001: val_acc improved from -inf to 0.76362, saving model to drive/My Drive/kaggle_nlp_tweet/best_val_acc.h5

Epoch 00001: acc improved from -inf to 0.73662, saving model to drive/My Drive/kaggle_nlp_tweet/best_acc.h5
Epoch 2/20
6090/6090 [==============================] - 67s 11ms/step - loss: 0.2920 - acc: 0.8851 - val_loss: 0.5701 - val_acc: 0.7479

Epoch 00002: val_acc did not improve from 0.76362

Epoch 00002: acc improved from 0.73662 to 0.88506, saving model to drive/My Drive/kaggle_nlp_tweet/best_acc.h5
Epoch 3/20
6090/6090 [==============================] - 67s 11ms/step - loss: 0.1585 - acc: 0.9429 - val_loss: 0.6058 - val_acc: 0.7505

Epoch 00003: val_acc did not improve from 0.76362

Epoch 00003: acc improved from 0.88506 to 0.94286, saving model to drive/My Drive/kaggle_nlp_tw

In [0]:
model.save_weights(os.path.join(mydir,'nlp.h5'))

## Predict on test set and write to output

In [0]:
test = pd.read_csv(os.path.join(mydir,'test.csv'))
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [0]:
tweet_test = []
ID = []
max_length_test = 0
for _, row in test.iterrows():
  ID.append(row['id'])
  text = row['text']
  text = text + ' ' + str(row['keyword'])
  text = text + ' ' + str(row['location'])
  # We would also remove any links or tags
  list_word = text.split()
  new_list = [word for word in list_word if ('http' not in word) and ('@' not in word) ]
  max_length_test = max([max_length_test,len(new_list)-1])
  tweet_test.append(' '.join(new_list))

print("MAX STRING LENGTH ", max_length_test)

MAX STRING LENGTH  35


In [0]:
# Predict
X_test = token.texts_to_sequences(tweet_test)
X_test = pad_sequences(X_test,maxlen=max_length)
X_test = np.asarray(X_test)

Y_pred = model.predict(X_test)



In [0]:
result = [[ID[i],int(Y_pred[i]+0.5)] for i in range(len(ID))]
result = pd.DataFrame(result, columns = ['id', 'target'])
result
result.to_csv(os.path.join(mydir,'submission_1.csv'),index = False)